You need to make a copy to your own Google drive if you want to edit the original notebook! Start by opening this notebook on Colab 👇

<a href="https://colab.research.google.com/github/PJalgotrader/Machine_Learning-USU/blob/fall22/Lectures/Class13-LogisticRegression%20(Python)/Multiclass%20classification/logistic_regression_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 



![logo](https://upload.wikimedia.org/wikipedia/commons/4/44/Huntsman-Wordmark-with-USU-Blue.gif#center) 



## 🔗 Links
[![Youtube](https://img.shields.io/badge/youtube_channel-1DA1F2?style=for-the-badge&logo=youtube&logoColor=white&color=red)](https://www.youtube.com/channel/UCNDElcuuyX-2pSatVBDpJJQ)
[![Profile](https://img.shields.io/badge/profiole-000?style=for-the-badge&logo=ko-fi&logoColor=white)](https://huntsman.usu.edu/directory/jahangiry-pedram)
[![linkedin](https://img.shields.io/badge/linkedin-0A66C2?style=for-the-badge&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/pedram-jahangiry-cfa-5778015a)

---

# Logistic Regression for multiclass classification

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()  #if you want to use seaborn themes with matplotlib functions
import warnings
warnings.filterwarnings('ignore')

In [2]:
rand_state= 1000

In [38]:
!wget -nc 'https://raw.githubusercontent.com/PJalgotrader/Machine_Learning-USU/fall22/data/supervised_mall_customers.csv'
df = pd.read_csv('supervised_mall_customers.csv')

--2022-10-11 21:36:51--  https://raw.githubusercontent.com/PJalgotrader/Machine_Learning-USU/fall22/data/supervised_mall_customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5080 (5.0K) [text/plain]
Saving to: ‘supervised_mall_customers.csv’

supervised_mall_cus 100%[===================>]   4.96K  --.-KB/s    in 0s      

2022-10-11 21:36:51 (34.8 MB/s) - ‘supervised_mall_customers.csv’ saved [5080/5080]



In [39]:
df.head()

,Unnamed: 0,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100),segments
0,0,1,Male,19,15,39,4
1,1,2,Male,21,15,81,4
2,2,3,Female,20,16,6,3
3,3,4,Female,23,16,77,4
4,4,5,Female,31,17,40,3


In [6]:
df.drop(['Unnamed: 0', 'CustomerID'], axis=1, inplace=True)

In [7]:
df.head()

,Genre,Age,Annual Income (k$),Spending Score (1-100),segments
0,Male,19,15,39,4
1,Male,21,15,81,4
2,Female,20,16,6,3
3,Female,23,16,77,4
4,Female,31,17,40,3


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Genre                   200 non-null    object
 1   Age                     200 non-null    int64 
 2   Annual Income (k$)      200 non-null    int64 
 3   Spending Score (1-100)  200 non-null    int64 
 4   segments                200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [9]:
# first we look at target variable proportions:
pd.crosstab(df['segments'],df['segments'], normalize='all')*100

segments,0,1,2,3,4,5
segments,,,,,,
0,22.5,0.0,0.0,0.0,0.0,0.0
1,0.0,19.5,0.0,0.0,0.0,0.0
2,0.0,0.0,19.0,0.0,0.0,0.0
3,0.0,0.0,0.0,10.5,0.0,0.0
4,0.0,0.0,0.0,0.0,11.5,0.0
5,0.0,0.0,0.0,0.0,0.0,17.0


## Data preprocessing 

In [10]:
df.isna().sum()  

Genre                     0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
segments                  0
dtype: int64

In [11]:
df = pd.get_dummies(df, drop_first=True)

In [12]:
df.head()

,Age,Annual Income (k$),Spending Score (1-100),segments,Genre_Male
0,19,15,39,4,1
1,21,15,81,4,1
2,20,16,6,3,0
3,23,16,77,4,0
4,31,17,40,3,0


In [13]:
df.rename(columns={"Genre_Male":"male"}, inplace=True)
df.head()

,Age,Annual Income (k$),Spending Score (1-100),segments,male
0,19,15,39,4,1
1,21,15,81,4,1
2,20,16,6,3,0
3,23,16,77,4,0
4,31,17,40,3,0


### Defining variables and splitting the data

In [14]:
y = df['segments']
X = df.drop('segments', axis=1) # becareful inplace= False

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)

In [15]:
X_train.head()

,Age,Annual Income (k$),Spending Score (1-100),male
132,25,72,34,0
167,33,86,95,0
24,54,28,14,0
125,31,70,77,0
131,39,71,75,1


In [16]:
y_train.head()

132    2
167    1
24     3
125    1
131    1
Name: segments, dtype: int64

##  Logistic regressions with Sklearn

In [17]:
from sklearn.linear_model import LogisticRegression

In [29]:
# Fitting Logistic Regression to the Training set
logistic = LogisticRegression(penalty='none', multi_class='auto' )
logistic.fit(X_train, y_train)

LogisticRegression(penalty='none')

In [30]:
# Predicting the Test set probabilities and classes
y_hat      = logistic.predict(X_test)
# predicted probabilities are reported for all 6 classes using softmax function. 

In [31]:
np.round(logistic.predict_proba(X_test),3)

array([[1.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.998, 0.   , 0.   , 0.002, 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.063, 0.   , 0.   , 0.937, 0.   , 0.   ],
       [0.   , 0.   , 1.   , 0.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.555, 0.445, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   , 0.   , 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 1.   ],
       [0.   , 0.   , 0.   , 1.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.

### Performance metrics:

In [34]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

#### Confusion matrix

In [35]:
print(confusion_matrix(y_test, y_hat))

[[13  0  0  1  0  0]
 [ 0  9  0  0  0  0]
 [ 0  1  2  0  0  0]
 [ 1  0  0  3  0  0]
 [ 0  0  0  0  4  0]
 [ 0  0  0  0  0  6]]


#### Built-in classification report

In [36]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93        14
           1       0.90      1.00      0.95         9
           2       1.00      0.67      0.80         3
           3       0.75      0.75      0.75         4
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         6

    accuracy                           0.93        40
   macro avg       0.93      0.89      0.90        40
weighted avg       0.93      0.93      0.92        40



In [37]:
matthews_corrcoef(y_test,y_hat)

0.9039692294111373